## Data Cleaning

Before starting with analyzing the data, it is important for us to clean our dataset.

Importing necessary packages 

In [2]:
import pandas as pd
import numpy as np
import datetime
import sklearn
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

C:\Users\Akash\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Akash\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Importing dataset.

In [3]:
df=pd.read_csv('kidney_disease.csv')

In [31]:
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


Let's see what are the different columns in our dataset.

In [4]:
df.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

Here we can see that apart from nan values we also have '\t?' values which need to take 

In [95]:
df.shape

(400, 26)

Let's find out whether our data contains any missing data or not.

In [96]:
df.isnull().sum()

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

From the above code, we can understand that there are lots of missing values in almost all columns. It is difficult for us to proceed with analyzing these data if we don't handle these missing data on first place.

Let's find out what are different types of data in our dataset.

In [97]:
df.dtypes

id                  int64
age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv                object
wc                 object
rc                 object
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

We can see that many colummns contains data which are not in either float or integer format. This can hinder our analysis. So, it is advisable for us to change these to numeric data.

Checking columns of object types

In [15]:
df['classification'].unique()

array(['ckd', 'notckd'], dtype=object)

In [16]:
df['ane'].unique()

array(['no', 'yes', nan], dtype=object)

In [17]:
df['pe'].unique()

array(['no', 'yes', nan], dtype=object)

In [18]:
df['appet'].unique()

array(['good', 'poor', nan], dtype=object)

In [19]:
df['cad'].unique()

array(['no', 'yes', nan], dtype=object)

In [20]:
df['dm'].unique()

array(['yes', 'no', nan], dtype=object)

In [21]:
df['htn'].unique()

array(['yes', 'no', nan], dtype=object)

In [22]:
df['rc'].unique()

array(['5.2', nan, '3.9', '4.6', '4.4', '5', '4', '3.7', '3.8', '3.4',
       '2.6', '2.8', '4.3', '3.2', '3.6', '4.1', '4.9', '2.5', '4.2',
       '4.5', '3.1', '4.7', '3.5', '6', '2.1', '5.6', '2.3', '2.9', '2.7',
       '8', '3.3', '3', '2.4', '4.8', '\t?', '5.4', '6.1', '6.2', '6.3',
       '5.1', '5.8', '5.5', '5.3', '6.4', '5.7', '5.9', '6.5'], dtype=object)

In [23]:
df['wc'].unique()

array(['7800', '6000', '7500', '6700', '7300', nan, '6900', '9600',
       '12100', '4500', '12200', '11000', '3800', '11400', '5300', '9200',
       '6200', '8300', '8400', '10300', '9800', '9100', '7900', '6400',
       '8600', '18900', '21600', '4300', '8500', '11300', '7200', '7700',
       '14600', '6300', '\t6200', '7100', '11800', '9400', '5500', '5800',
       '13200', '12500', '5600', '7000', '11900', '10400', '10700',
       '12700', '6800', '6500', '13600', '10200', '9000', '14900', '8200',
       '15200', '5000', '16300', '12400', '\t8400', '10500', '4200',
       '4700', '10900', '8100', '9500', '2200', '12800', '11200', '19100',
       '\t?', '12300', '16700', '2600', '26400', '8800', '7400', '4900',
       '8000', '12000', '15700', '4100', '5700', '11500', '5400', '10800',
       '9900', '5200', '5900', '9300', '9700', '5100', '6600'], dtype=object)

In [24]:
df['pcv'].unique()

array(['44', '38', '31', '32', '35', '39', '36', '33', '29', '28', nan,
       '16', '24', '37', '30', '34', '40', '45', '27', '48', '\t?', '52',
       '14', '22', '18', '42', '17', '46', '23', '19', '25', '41', '26',
       '15', '21', '43', '20', '\t43', '47', '9', '49', '50', '53', '51',
       '54'], dtype=object)

In [26]:
df['ba'].unique()

array(['notpresent', 'present', nan], dtype=object)

In [27]:
df['pcc'].unique()

array(['notpresent', 'present', nan], dtype=object)

In [28]:
df['pc'].unique()

array(['normal', 'abnormal', nan], dtype=object)

In [29]:
df['rbc'].unique()

array([nan, 'normal', 'abnormal'], dtype=object)

This shows us that we need to handle '\t?' and also type cast few columns 

We found that there are special characters in our dataset. This can hinder our analysis. So, replacing it with `null`

In [99]:
df.replace(to_replace="\t?",value=np.nan,inplace=True)

In [100]:
df.dtypes

id                  int64
age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv                object
wc                 object
rc                 object
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

We have to handle missing values. We analyzed our dataste throughly and we can up with few reults.

1) In columns like: age, bgr, bu, sc, sod, pot, hemo, pcv and rc , missing values will be replaced with average/mean of that column's data.

2) In columns like: rbc, pc, pcc, ba, pcv, wc, rc, htn, dm , cad, appet, pe, ane and classification , missing values will be replaced with most frequently occuring data as these contain string data.

In [101]:
bp = pd.DataFrame(df.groupby('bp').size()).idxmax()[0]
df['bp'] = df['bp'].fillna(bp)

sg = pd.DataFrame(df.groupby('sg').size()).idxmax()[0]
df['sg'] = df['sg'].fillna(sg)

al = pd.DataFrame(df.groupby('al').size()).idxmax()[0]
df['al'] = df['al'].fillna(al)

su = pd.DataFrame(df.groupby('su').size()).idxmax()[0]
df['su'] = df['su'].fillna(su)

rbc = pd.DataFrame(df.groupby('rbc').size()).idxmax()[0]
df['rbc'] = df['rbc'].fillna(rbc)

pc = pd.DataFrame(df.groupby('pc').size()).idxmax()[0]
df['pc'] = df['pc'].fillna(pc)

pcc = pd.DataFrame(df.groupby('pcc').size()).idxmax()[0]
df['pcc'] = df['pcc'].fillna(pcc)

ba = pd.DataFrame(df.groupby('ba').size()).idxmax()[0]
df['ba'] = df['ba'].fillna(ba)

bgr = pd.DataFrame(df.groupby('bgr').size()).idxmax()[0]
df['bgr'] = df['bgr'].fillna(bgr)

wc = pd.DataFrame(df.groupby('wc').size()).idxmax()[0]
df['wc'] = df['wc'].fillna(wc)

htn = pd.DataFrame(df.groupby('htn').size()).idxmax()[0]
df['htn'] = df['htn'].fillna(htn)

dm = pd.DataFrame(df.groupby('dm').size()).idxmax()[0]
df['dm'] = df['dm'].fillna(dm)

wc = pd.DataFrame(df.groupby('wc').size()).idxmax()[0]
df['wc'] = df['wc'].fillna(wc)

cad = pd.DataFrame(df.groupby('cad').size()).idxmax()[0]
df['cad'] = df['cad'].fillna(cad)

appet = pd.DataFrame(df.groupby('appet').size()).idxmax()[0]
df['appet'] = df['appet'].fillna(appet)

pe = pd.DataFrame(df.groupby('pe').size()).idxmax()[0]
df['pe'] = df['pe'].fillna(pe)

ane = pd.DataFrame(df.groupby('ane').size()).idxmax()[0]
df['ane'] = df['ane'].fillna(ane)

classification = pd.DataFrame(df.groupby('classification').size()).idxmax()[0]
df['classification'] = df['classification'].fillna(classification)

Type casting the columns into numeric data type

In [102]:
df['rc'] = df['rc'].astype('float64') 
df['pcv'] = df['pcv'].astype('float64')
df['wc'] = df['wc'].astype('int64')

In [103]:
df['age'] = df['age'].fillna(df['age'].mean(axis=0))
df['bgr'] = df['bgr'].fillna(df['bgr'].mean(axis=0))
df['sc'] = df['sc'].fillna(df['sc'].mean(axis=0))
df['bu'] = df['bu'].fillna(df['bu'].mean(axis=0))
df['sod'] = df['sod'].fillna(df['sod'].mean(axis=0))
df['pot'] = df['pot'].fillna(df['pot'].mean(axis=0))
df['pcv'] = df['pcv'].fillna(df['pcv'].mean(axis=0))
df['hemo'] = df['hemo'].fillna(df['hemo'].mean(axis=0))
df['rc'] = df['rc'].fillna(df['rc'].mean(axis=0))

In [104]:
df.dtypes

id                  int64
age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv               float64
wc                  int64
rc                float64
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

Rounding off to nearest decimal value.

In [105]:
df.age = df.age.round()
df.bu = df.bu.round()
df.sod = df.sod.round()
df.pcv = df.pcv.round()

In [106]:
df.head(10)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,normal,normal,notpresent,notpresent,...,44.0,7800,5.200000,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,normal,normal,notpresent,notpresent,...,38.0,6000,4.707435,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31.0,7500,4.707435,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32.0,6700,3.900000,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35.0,7300,4.600000,no,no,no,good,no,no,ckd
5,5,60.0,90.0,1.015,3.0,0.0,normal,normal,notpresent,notpresent,...,39.0,7800,4.400000,yes,yes,no,good,yes,no,ckd
6,6,68.0,70.0,1.010,0.0,0.0,normal,normal,notpresent,notpresent,...,36.0,9800,4.707435,no,no,no,good,no,no,ckd
7,7,24.0,80.0,1.015,2.0,4.0,normal,abnormal,notpresent,notpresent,...,44.0,6900,5.000000,no,yes,no,good,yes,no,ckd
8,8,52.0,100.0,1.015,3.0,0.0,normal,abnormal,present,notpresent,...,33.0,9600,4.000000,yes,yes,no,good,no,yes,ckd
9,9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,...,29.0,12100,3.700000,yes,yes,no,poor,no,yes,ckd


In [107]:
df.isnull().sum()

id                0
age               0
bp                0
sg                0
al                0
su                0
rbc               0
pc                0
pcc               0
ba                0
bgr               0
bu                0
sc                0
sod               0
pot               0
hemo              0
pcv               0
wc                0
rc                0
htn               0
dm                0
cad               0
appet             0
pe                0
ane               0
classification    0
dtype: int64

Now, we can see that we have removed all null values. But we still have `string` data in our which has to be removed. For this, we are creating dummy variables. 

In [108]:
df=pd.get_dummies(df,prefix=['RBC','PC','PCC' , 'BA' , 'HTN' , 'DM' , 'CAD' , 'Appet' , 'PE' , 'Ane' , 'Classification'],columns=['rbc','pc','pcc' , 'ba' , 'htn' , 'dm' , 'cad' , 'appet' , 'pe' , 'ane' , 'classification'])

In [109]:
df.isnull().sum()

id                       0
age                      0
bp                       0
sg                       0
al                       0
su                       0
bgr                      0
bu                       0
sc                       0
sod                      0
pot                      0
hemo                     0
pcv                      0
wc                       0
rc                       0
RBC_abnormal             0
RBC_normal               0
PC_abnormal              0
PC_normal                0
PCC_notpresent           0
PCC_present              0
BA_notpresent            0
BA_present               0
HTN_no                   0
HTN_yes                  0
DM_no                    0
DM_yes                   0
CAD_no                   0
CAD_yes                  0
Appet_good               0
Appet_poor               0
PE_no                    0
PE_yes                   0
Ane_no                   0
Ane_yes                  0
Classification_ckd       0
Classification_notckd    0
d

In [110]:
df.head()

,id,age,bp,sg,al,su,bgr,bu,sc,sod,...,CAD_no,CAD_yes,Appet_good,Appet_poor,PE_no,PE_yes,Ane_no,Ane_yes,Classification_ckd,Classification_notckd
0,0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,138.0,...,1,0,1,0,1,0,1,0,1,0
1,1,7.0,50.0,1.020,4.0,0.0,99.0,18.0,0.8,138.0,...,1,0,1,0,1,0,1,0,1,0
2,2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,138.0,...,1,0,0,1,1,0,0,1,1,0
3,3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,...,1,0,0,1,0,1,0,1,1,0
4,4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,138.0,...,1,0,1,0,1,0,1,0,1,0


Removing all unwanted columns

In [111]:
df.drop(['RBC_abnormal'],axis=1,inplace=True)
df.drop(['PC_abnormal'],axis=1,inplace=True)
df.drop(['PCC_notpresent'],axis=1,inplace=True)
df.drop(['BA_notpresent'],axis=1,inplace=True)
df.drop(['HTN_no'],axis=1,inplace=True)
df.drop(['DM_no'],axis=1,inplace=True)
df.drop(['CAD_no'],axis=1,inplace=True)
df.drop(['Appet_poor'],axis=1,inplace=True)
df.drop(['PE_no'],axis=1,inplace=True)
df.drop(['Ane_no'],axis=1,inplace=True)
df.drop(['Classification_notckd'],axis=1,inplace=True)
df.drop(['id'],axis=1,inplace=True)

Saving the cleaned data in a new `csv` file.

In [114]:
df.to_csv('ckd.csv' , index = False)

Conclusion: We have handled all the missing data from our dataset and we have also removed all the string values and converted it to numeric data which can make our data analysis easy.